In [1]:
import os
import cv2
import numpy as np
import fitz
from PIL import Image

In [2]:
def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)

    # Apply adaptive thresholding to enhance contrast
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)

    return thresh

def enlarge_image(image, scale_factor=3):
    # Enlarge the image with Lanczos interpolation
    enlarged_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LANCZOS4)
    
    return enlarged_image

def enhance_quality(image):
    # Sharpen the image
    sharpened = cv2.filter2D(image, -1, np.array([[-1, -1, -1],
                                                  [-1,  9, -1],
                                                  [-1, -1, -1]]))
    
    # Denoise the image
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    
    return denoised


def extract_alphabets(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    pdf_document = fitz.open(pdf_path)
    x_coords = []
    y_coords = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        page_image = page.get_pixmap()
        np_page_image = np.frombuffer(page_image.samples, dtype=np.uint8).reshape((page_image.height, page_image.width, page_image.n))

        # Preprocess image
        processed_image = preprocess_image(np_page_image)

        # Find contours
        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # Get bounding box of each contour
            x, y, w, h = cv2.boundingRect(contour)

            x_coords.append(x)
            y_coords.append(y)

            # Crop, enlarge, and enhance quality of region containing alphabet
            alphabet_region = np_page_image[y:y+h, x:x+w]
            enlarged_region = enlarge_image(alphabet_region)
            enhanced_region = enhance_quality(enlarged_region)

            # Save region containing alphabet
            alphabet_image = Image.fromarray(enhanced_region)
            alphabet_image.save(f"{output_folder}/alphabet_{page_num}_{x}_{y}.png")

# pdf_path = "semi_book.pdf"
# output_folder = "segmented_data"
# extract_alphabets(pdf_path, output_folder)

pdf_path = "adv.pdf"
output_folder = "adv"
extract_alphabets(pdf_path, output_folder)

In [3]:
import os
import cv2
import fitz
import numpy as np
from PIL import Image

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)
    return thresh

def enlarge_image(image, scale_factor=3):
    enlarged_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

def enhance_quality(image):
    sharpened = cv2.filter2D(image, -1, np.array([[-1, -1, -1], [-1,  9, -1], [-1, -1, -1]]))
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    return denoised

def extract_alphabets(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    sorted_x_folder = os.path.join(output_folder, "sorted_x")
    sorted_y_folder = os.path.join(output_folder, "sorted_y")
    
    os.makedirs(sorted_x_folder, exist_ok=True)
    os.makedirs(sorted_y_folder, exist_ok=True)

    pdf_document = fitz.open(pdf_path)
    coordinates = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_image = page.get_pixmap()
        np_page_image = np.frombuffer(page_image.samples, dtype=np.uint8).reshape((page_image.height, page_image.width, page_image.n))

        processed_image = preprocess_image(np_page_image)

        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            coordinates.append((page_num, x, y, w, h))

            alphabet_region = np_page_image[y:y+h, x:x+w]
            enlarged_region = enlarge_image(alphabet_region)
            enhanced_region = enhance_quality(enlarged_region)

            alphabet_image = Image.fromarray(enhanced_region)
            alphabet_image.save(f"{output_folder}/alphabet_{page_num}_{x}__{y}__{w}__{h}.png")

    coordinates_sorted_by_x = sorted(coordinates, key=lambda item: item[1])
    coordinates_sorted_by_y = sorted(coordinates, key=lambda item: item[2])

    print("Coordinates sorted by x:")
    for coord in coordinates_sorted_by_x:
        print(coord)

    print("\nCoordinates sorted by y:")
    for coord in coordinates_sorted_by_y:
        print(coord)

    for i, (page_num, x, y, w, h) in enumerate(coordinates_sorted_by_x):
        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)
        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(f"{sorted_x_folder}/alphabet_{i}_page{page_num}_{x}__{y}__{w}__{h}.png")

    for i, (page_num, x, y, w, h) in enumerate(coordinates_sorted_by_y):
        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)
        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(f"{sorted_y_folder}/alphabet_{i}_page{page_num}_{x}__{y}__{w}__{h}.png")

pdf_path = "Analysis/yaman_kalyaan_3_taal.pdf"
output_folder = "Analysis/yaman_kalyaan_3_taal_coordinates"
extract_alphabets(pdf_path, output_folder)


Coordinates sorted by x:
(0, 146, 638, 13, 9)
(0, 147, 576, 9, 10)
(0, 147, 453, 9, 10)
(0, 148, 690, 7, 7)
(0, 148, 651, 9, 11)
(0, 148, 612, 10, 12)
(0, 148, 591, 8, 10)
(0, 148, 527, 14, 12)
(0, 148, 489, 9, 10)
(0, 149, 670, 13, 14)
(0, 149, 546, 8, 15)
(0, 149, 257, 12, 24)
(0, 150, 629, 6, 5)
(0, 150, 567, 7, 6)
(0, 150, 505, 7, 7)
(0, 150, 317, 9, 10)
(0, 151, 467, 8, 11)
(0, 152, 308, 7, 6)
(0, 152, 247, 7, 6)
(0, 153, 371, 7, 6)
(0, 153, 208, 10, 11)
(0, 154, 332, 10, 10)
(0, 154, 287, 9, 15)
(0, 154, 230, 10, 14)
(0, 155, 354, 12, 10)
(0, 157, 518, 14, 5)
(0, 158, 583, 12, 5)
(0, 163, 636, 7, 10)
(0, 165, 649, 9, 13)
(0, 165, 486, 12, 16)
(0, 167, 591, 9, 10)
(0, 167, 529, 9, 10)
(0, 167, 467, 9, 11)
(0, 168, 673, 7, 10)
(0, 168, 612, 7, 10)
(0, 168, 550, 7, 11)
(0, 171, 332, 10, 10)
(0, 171, 273, 9, 5)
(0, 172, 354, 7, 11)
(0, 172, 231, 9, 10)
(0, 172, 203, 8, 16)
(0, 174, 291, 7, 11)
(0, 179, 649, 13, 12)
(0, 180, 673, 9, 11)
(0, 183, 612, 9, 11)
(0, 183, 529, 9, 11)
(0, 18

In [7]:
import os
import cv2
import fitz
import numpy as np
from PIL import Image

# Preprocess the input image for model prediction
def preprocess_image_for_prediction(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (32, 32))  # Resize to match the model's input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    return image

# Predict the class of the image using the trained model
def predict_class(image):
    preprocessed_image = preprocess_image_for_prediction(image)
    predictions = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predictions, axis=1)
    return predicted_class_index[0]

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)
    return thresh

def enlarge_image(image, scale_factor=3):
    enlarged_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

def enhance_quality(image):
    sharpened = cv2.filter2D(image, -1, np.array([[-1, -1, -1], [-1,  9, -1], [-1, -1, -1]]))
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    return denoised

def extract_alphabets(pdf_path, output_folder, aspect_ratio_threshold=10):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    row_wise_folder = os.path.join(output_folder, "row_wise")
    special_class_folder = os.path.join(output_folder, "special_class")
    os.makedirs(row_wise_folder, exist_ok=True)
    os.makedirs(special_class_folder, exist_ok=True)

    pdf_document = fitz.open(pdf_path)
    coordinates = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_image = page.get_pixmap()
        np_page_image = np.frombuffer(page_image.samples, dtype=np.uint8).reshape((page_image.height, page_image.width, page_image.n))

        processed_image = preprocess_image(np_page_image)

        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            coordinates.append((page_num, x, y, w, h))

    coordinates_sorted_by_y = sorted(coordinates, key=lambda item: item[2])

    print("Coordinates sorted by y:")
    for coord in coordinates_sorted_by_y:
        print(coord)

    row_number = 1
    limit = 0

    for i, (page_num, x, y, w, h) in enumerate(coordinates_sorted_by_y):
        # Ignore vertical lines based on aspect ratio
        aspect_ratio_threshold = 3
        if h / w > aspect_ratio_threshold:
            continue

        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)

        if i == 0:
            limit = y + int(h / 2)
        elif y > limit:
            row_number += 1
            limit = y + int(h / 2)
        else:
            limit = max(limit, y + int(h / 2))

        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(f"{row_wise_folder}/row{row_number}_x{x}_w{w}_page{page_num}_{x}_{y}.png")

pdf_path = "adv.pdf"
output_folder = "adv3"
extract_alphabets(pdf_path, output_folder)


Coordinates sorted by y:
(0, 321, 69, 7, 17)
(0, 271, 69, 6, 16)
(0, 300, 71, 8, 12)
(0, 292, 72, 6, 11)
(0, 280, 107, 42, 14)
(0, 396, 129, 5, 168)
(0, 443, 130, 14, 14)
(0, 350, 131, 10, 13)
(0, 312, 131, 10, 13)
(0, 185, 131, 13, 14)
(0, 291, 132, 18, 172)
(0, 406, 133, 8, 11)
(0, 211, 133, 7, 171)
(0, 132, 135, 10, 10)
(0, 463, 145, 19, 15)
(0, 356, 145, 9, 18)
(0, 313, 145, 15, 13)
(0, 422, 146, 26, 20)
(0, 405, 146, 14, 17)
(0, 376, 146, 15, 16)
(0, 335, 146, 11, 17)
(0, 266, 146, 16, 13)
(0, 286, 147, 14, 16)
(0, 186, 148, 24, 11)
(0, 242, 149, 20, 10)
(0, 451, 150, 9, 13)
(0, 219, 150, 20, 9)
(0, 169, 150, 14, 10)
(0, 149, 151, 17, 10)
(0, 134, 151, 9, 13)
(0, 314, 162, 15, 7)
(0, 466, 163, 14, 5)
(0, 266, 163, 15, 6)
(0, 244, 163, 17, 7)
(0, 188, 163, 17, 7)
(0, 221, 164, 16, 6)
(0, 170, 164, 14, 6)
(0, 152, 164, 12, 6)
(0, 338, 172, 13, 20)
(0, 378, 173, 13, 18)
(0, 357, 173, 14, 18)
(0, 313, 174, 16, 12)
(0, 468, 176, 14, 12)
(0, 134, 176, 14, 16)
(0, 450, 177, 8, 13)
(0, 43

In [13]:
import os
import cv2
import fitz
import numpy as np
from PIL import Image

# Preprocess the input image for model prediction
def preprocess_image_for_prediction(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (32, 32))  # Resize to match the model's input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    return image

# Preprocess the input image
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)
    return thresh

def enlarge_image(image, scale_factor=3):
    enlarged_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

def enhance_quality(image):
    sharpened = cv2.filter2D(image, -1, np.array([[-1, -1, -1], [-1,  9, -1], [-1, -1, -1]]))
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    return denoised

def extract_alphabets(pdf_path, output_folder, aspect_ratio_threshold=3):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    row_wise_folder = os.path.join(output_folder, "row_wise")
    column_wise_folder = os.path.join(output_folder, "column_wise")
    special_class_folder = os.path.join(output_folder, "special_class")
    os.makedirs(row_wise_folder, exist_ok=True)
    os.makedirs(column_wise_folder, exist_ok=True)
    os.makedirs(special_class_folder, exist_ok=True)

    pdf_document = fitz.open(pdf_path)
    coordinates = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_image = page.get_pixmap()
        np_page_image = np.frombuffer(page_image.samples, dtype=np.uint8).reshape((page_image.height, page_image.width, page_image.n))

        processed_image = preprocess_image(np_page_image)

        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            coordinates.append((page_num, x, y, w, h))

    coordinates_sorted_by_y = sorted(coordinates, key=lambda item: item[2])
    coordinates_sorted_by_x = sorted(coordinates, key=lambda item: item[1])

    print("Coordinates sorted by y:")
    for coord in coordinates_sorted_by_y:
        print(coord)

    print("Coordinates sorted by x:")
    for coord in coordinates_sorted_by_x:
        print(coord)

    row_number = 1
    row_limit = 0

    for i, (page_num, x, y, w, h) in enumerate(coordinates_sorted_by_y):
        # Ignore vertical lines based on aspect ratio
        if h / w > aspect_ratio_threshold:
            continue

        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)

        if i == 0:
            row_limit = y + int(h / 2)
        elif y > row_limit:
            row_number += 1
            row_limit = y + int(h / 2)
        else:
            row_limit = max(row_limit, y + int(h / 2))

        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(f"{row_wise_folder}/row{row_number}_x{x}_w{w}_page{page_num}_{x}_{y}.png")

    col_number = 1
    col_limit = 0

    for i, (page_num, x, y, w, h) in enumerate(coordinates_sorted_by_x):
        # Ignore vertical lines based on aspect ratio
        if h / w > aspect_ratio_threshold:
            continue

        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)

        # Check the width-to-height ratio
        if w / h > aspect_ratio_threshold:
            alphabet_image = Image.fromarray(enlarged_region)
            alphabet_image.save(f"{column_wise_folder}/col{col_number}_x{x}_w{w}_page{page_num}_{x}_{y}.png")
            continue  # Assign same column number and continue without updating limit

        if i == 0:
            col_limit = x + int(w / 2)
        elif x > col_limit:
            col_number += 1
            col_limit = x + int(w / 2)
        else:
            col_limit = max(col_limit, x + int(w / 2))

        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(f"{column_wise_folder}/col{col_number}_x{x}_w{w}_page{page_num}_{x}_{y}.png")

pdf_path = "khamaj_3_taal_2.pdf"
output_folder = "khamaj_3_taal_2"
extract_alphabets(pdf_path, output_folder)


Coordinates sorted by y:
(0, 342, 55, 6, 18)
(0, 277, 55, 6, 17)
(0, 319, 56, 8, 15)
(0, 302, 56, 14, 16)
(0, 270, 92, 47, 17)
(0, 322, 93, 7, 19)
(0, 383, 94, 12, 18)
(0, 370, 97, 8, 12)
(0, 334, 97, 35, 12)
(0, 227, 98, 42, 14)
(0, 286, 137, 41, 20)
(0, 327, 152, 6, 6)
(0, 411, 182, 6, 265)
(0, 257, 182, 8, 17)
(0, 303, 184, 6, 265)
(0, 195, 184, 6, 268)
(0, 204, 186, 8, 12)
(0, 500, 203, 13, 16)
(0, 427, 204, 12, 15)
(0, 390, 204, 12, 15)
(0, 342, 204, 11, 15)
(0, 319, 204, 12, 15)
(0, 260, 205, 11, 15)
(0, 232, 205, 12, 15)
(0, 211, 205, 11, 15)
(0, 178, 205, 11, 15)
(0, 155, 205, 12, 15)
(0, 132, 205, 16, 14)
(0, 107, 206, 17, 13)
(0, 477, 209, 11, 5)
(0, 450, 209, 13, 5)
(0, 367, 209, 12, 6)
(0, 282, 211, 12, 5)
(0, 346, 229, 9, 22)
(0, 426, 230, 16, 20)
(0, 257, 230, 17, 22)
(0, 135, 230, 10, 22)
(0, 477, 235, 10, 14)
(0, 453, 235, 10, 14)
(0, 370, 235, 10, 14)
(0, 499, 236, 11, 15)
(0, 390, 236, 12, 19)
(0, 319, 236, 13, 15)
(0, 178, 236, 11, 15)
(0, 285, 237, 10, 14)
(0, 236, 

In [2]:
import os
import cv2
import fitz
import numpy as np
from PIL import Image
import bisect

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)
    
    return thresh

def enlarge_image(image, scale_factor=3):
    enlarged_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LANCZOS4)
    return enlarged_image

def enhance_quality(image):
    sharpened = cv2.filter2D(image, -1, np.array([[-1, -1, -1], [-1,  9, -1], [-1, -1, -1]]))
    denoised = cv2.fastNlMeansDenoisingColored(sharpened, None, 10, 10, 7, 21)
    return denoised

def create_mapping(coordinates, aspect_ratio_threshold, is_row=True):
    mapping = []
    number = 1

    for i, (page_num, x, y, w, h) in enumerate(coordinates):
        if is_row:
            if h / w > aspect_ratio_threshold:
                continue
            coord = y
            size = h
        else:
            if h / w > aspect_ratio_threshold:
                continue

            if w / h > aspect_ratio_threshold:
                continue
            coord = x
            size = w

        if i == 0:
            upper_limit = coord + int(size / 2)
            lower_limit = coord
            mapping.append((number, lower_limit, upper_limit))
        elif coord > mapping[-1][2]:
            number += 1
            lower_limit = coord
            upper_limit = coord + int(size / 2)
            mapping.append((number, lower_limit, upper_limit))
        else:
            upper_limit = max(mapping[-1][2], coord + int(size / 2))
            mapping[-1] = (number, mapping[-1][1], upper_limit)

    return mapping

def assign_number(coord, mapping, is_row=True):
    for num, lower_limit, upper_limit in mapping:
        if lower_limit <= coord <= upper_limit:
            return num

    # If no appropriate number found, find the maximum upper limit that is less than the given coordinate
    upper_limits = [upper_limit for _, _, upper_limit in mapping]
    index = bisect.bisect_right(upper_limits, coord) - 1
    
    if index >= 0:
        return mapping[index][0]
    else:
        return -1


def extract_alphabets(pdf_path, output_folder, aspect_ratio_threshold=3):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    pdf_document = fitz.open(pdf_path)
    coordinates = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_image = page.get_pixmap()
        np_page_image = np.frombuffer(page_image.samples, dtype=np.uint8).reshape((page_image.height, page_image.width, page_image.n))

        processed_image = preprocess_image(np_page_image)

        contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            coordinates.append((page_num, x, y, w, h))

    coordinates_sorted_by_y = sorted(coordinates, key=lambda item: item[2])
    coordinates_sorted_by_x = sorted(coordinates, key=lambda item: item[1])

    row_mapping = create_mapping(coordinates_sorted_by_y, aspect_ratio_threshold, is_row=True)
    col_mapping = create_mapping(coordinates_sorted_by_x, aspect_ratio_threshold, is_row=False)

    print("Row mapping:", row_mapping)
    print("Column mapping:", col_mapping)

    for page_num, x, y, w, h in coordinates:

        if h / w > aspect_ratio_threshold:
            continue

        row_num = assign_number(y, row_mapping, is_row=True)
        col_num = assign_number(x, col_mapping, is_row=False)

        if row_num == -1 or col_num == -1:
            continue

        alphabet_region = np_page_image[y:y+h, x:x+w]
        enlarged_region = enlarge_image(alphabet_region)
        enhanced_region = enhance_quality(enlarged_region)

        # Ensure unique filenames
        base_filename = f"{page_num}_row{row_num}_col{col_num}"
        counter = 1
        filename = f"{base_filename}.png"
        while os.path.exists(os.path.join(output_folder, filename)):
            filename = f"{base_filename}_{counter}.png"
            counter += 1

        alphabet_image = Image.fromarray(enhanced_region)
        alphabet_image.save(os.path.join(output_folder, filename))

pdf_path = "Analysis/alhaiya_bilawal_3_taal.pdf"
output_folder = "Analysis/alhaiya_bilawal_3_taal"
extract_alphabets(pdf_path, output_folder)


Row mapping: [(1, 93, 107), (2, 133, 150), (3, 182, 198), (4, 228, 374), (5, 379, 392), (6, 417, 458), (7, 473, 496), (8, 522, 531), (9, 547, 715), (10, 739, 766)]
Column mapping: [(1, 75, 443), (2, 454, 490), (3, 492, 509)]
